In [ ]:
# Rate per session for operated flight (based on DEPARTURE_TIME)

# Table with these columns, values = ratio of operated flight

# Origin airport
# session1: 0 - 6am ratio -> 00:00 - 05:59
# session2: 6am - 12noon -> 06:00 - 11:59
# session3: 12noon - 6pm -> 12:00 - 17:59
# session4: 6pm - midnight -> 18:00 - 23:59

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_flights = pd.read_csv('flights.csv')

/var/folders/sp/v99xtfrs4p35rhqv158v85h80000gn/T/ipykernel_2820/772266734.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_flights = pd.read_csv('flights.csv')


In [4]:
# Set splitter every 6 hours (also can set every 1, 2, 3, 4, 6, 8, 12 hours)
session_split = 600

In [5]:
total_session = 2400 // session_split

In [6]:
df_flights['session'] = df_flights['DEPARTURE_TIME'].apply(lambda x: total_session if x == 2400 else x // session_split + 1)

In [7]:
df_flights['operated'] = df_flights['CANCELLED'].apply(lambda x: 1 if x == 0 else 0)

In [8]:
df_flights.sample(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,session,operated
1733763,2015,4,21,2,EV,5299,N604QX,MSP,RST,1300,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1
1097036,2015,3,13,5,MQ,3599,N817MQ,LGA,SDF,830,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1
969678,2015,3,5,4,AA,971,N013AA,DFW,ELP,1147,...,0,0,NaN,5.0,0.0,14.0,0.0,0.0,3.0,1
5261521,2015,11,25,3,AA,594,N753US,PHX,IAH,1701,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1
1009661,2015,3,7,6,WN,1654,N205WN,STL,MCO,2045,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1


In [9]:
for i in range(1, total_session + 1):
    df_flights[f'session{i}_count'] = (df_flights['session'] == i).astype(int)

In [10]:
df_flights.sample(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,session,operated,session1_count,session2_count,session3_count,session4_count
609404,2015,2,10,2,DL,2001,N928DN,LGA,ATL,1000,...,NaN,NaN,NaN,NaN,2.0,1,0,1,0,0
2005112,2015,5,8,5,US,1995,N582UW,DFW,PHL,825,...,NaN,NaN,NaN,NaN,2.0,1,0,1,0,0
2399191,2015,6,1,1,AA,1140,N3DVAA,LGA,DFW,1825,...,NaN,NaN,NaN,NaN,4.0,1,0,0,0,1
2672012,2015,6,18,4,OO,5604,N679SA,ACV,SFO,600,...,NaN,NaN,NaN,NaN,1.0,1,1,0,0,0
4247841,2015,9,22,2,WN,1583,N720WN,STL,TPA,915,...,NaN,NaN,NaN,NaN,2.0,1,0,1,0,0


In [11]:
df_session_list = list()

In [12]:
df_session_list.append(df_flights.groupby('ORIGIN_AIRPORT').agg(operated_flight_count = ('operated', 'sum')).reset_index())

In [13]:
df_session_list[0]

,ORIGIN_AIRPORT,operated_flight_count
0,10135,226
1,10136,182
2,10140,1706
3,10141,66
4,10146,82
...,...,...
925,WRG,653
926,WYS,208
927,XNA,8987
928,YAK,657


In [14]:
for i in range(1, total_session + 1):
    df = df_flights.groupby('ORIGIN_AIRPORT').agg({f'session{i}_count':'sum'}).reset_index()
    df_session_list.append(df)

In [15]:
df_session_list

[    ORIGIN_AIRPORT  operated_flight_count
 0            10135                    226
 1            10136                    182
 2            10140                   1706
 3            10141                     66
 4            10146                     82
 ..             ...                    ...
 925            WRG                    653
 926            WYS                    208
 927            XNA                   8987
 928            YAK                    657
 929            YUM                   1854
 
 [930 rows x 2 columns],
     ORIGIN_AIRPORT  session1_count
 0            10135              33
 1            10136              17
 2            10140              72
 3            10141              28
 4            10146              20
 ..             ...             ...
 925            WRG               0
 926            WYS               0
 927            XNA             737
 928            YAK               0
 929            YUM               0
 
 [930 rows x 2 columns]

In [16]:
df_output = df_session_list[0]

In [17]:
for i in range(1, total_session + 1):
    df_output = pd.merge(df_output, df_session_list[i], how = 'outer', on = 'ORIGIN_AIRPORT')

In [18]:
df_output.sample(5)

,ORIGIN_AIRPORT,operated_flight_count,session1_count,session2_count,session3_count,session4_count
298,15412,588,5,290,240,53
496,13342,146,7,65,57,17
535,14193,26,0,14,9,3
118,11921,257,0,148,79,30
25,10529,1616,128,685,596,207


In [19]:
for i in range(1, total_session + 1):
    df_output[f'session{i}_ratio'] = df_output[f'session{i}_count'] / df_output['operated_flight_count']

In [20]:
df_output.sample(5)

,ORIGIN_AIRPORT,operated_flight_count,session1_count,session2_count,session3_count,session4_count,session1_ratio,session2_ratio,session3_ratio,session4_ratio
839,OTH,265,0,1,260,4,0.000000,0.003774,0.981132,0.015094
463,12819,12,0,4,6,2,0.000000,0.333333,0.500000,0.166667
665,CEC,174,26,58,86,4,0.149425,0.333333,0.494253,0.022989
861,PWM,4543,605,1438,1905,597,0.133172,0.316531,0.419326,0.131411
745,GUM,331,0,329,2,1,0.000000,0.993958,0.006042,0.003021


In [21]:
final_output = df_output.drop(columns = 'operated_flight_count')

In [22]:
for i in range(1, total_session + 1):
    final_output = final_output.drop(columns = f'session{i}_count')

In [23]:
final_output.sample(5)

,ORIGIN_AIRPORT,session1_ratio,session2_ratio,session3_ratio,session4_ratio
744,GUC,0.000000,0.233051,0.735169,0.036017
615,ACV,0.190701,0.249015,0.381403,0.179669
835,ONT,0.099854,0.420038,0.303608,0.176950
874,RSW,0.041852,0.329374,0.416556,0.212515
632,AVP,0.205109,0.371901,0.380917,0.042074


In [24]:
!pip install plotly

In [25]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = 'iframe'

In [26]:
df_plot = final_output.tail(10)

In [27]:
session_ratio_list = []

In [28]:
for row in df_plot.iterrows():
    session_ratio_list.append(list(row[1]))

In [29]:
session_ratio_list

[['TYS',
  0.03136504196730967,
  0.48873509056103664,
  0.3619496392283905,
  0.11883375055220144],
 ['UST', 0.006944444444444444, 0.2777777777777778, 0.09027777777777778, 0.625],
 ['VEL', 0.0, 0.575, 0.17, 0.255],
 ['VLD',
  0.2513484358144552,
  0.3117583603020496,
  0.41423948220064727,
  0.022653721682847898],
 ['VPS',
  0.09750471797022436,
  0.3810023065632208,
  0.44181170056615643,
  0.07989096246592577],
 ['WRG', 0.0, 0.47320061255742724, 0.4946401225114854, 0.03215926493108729],
 ['WYS', 0.0, 0.0, 0.9567307692307693, 0.04326923076923077],
 ['XNA',
  0.08200734394124846,
  0.37710025592522534,
  0.40814509847557584,
  0.1338600200289307],
 ['YAK', 0.0, 0.4368340943683409, 0.0989345509893455, 0.4642313546423135],
 ['YUM', 0.0, 0.6073354908306364, 0.3106796116504854, 0.08198489751887811]]

In [42]:
fig = go.Figure()

In [43]:
for i in session_ratio_list:
    fig.add_trace(
        go.Scatter(
            x = [f"session{j}" for j in range(1, total_session + 1)],
            y = i[1::],
            mode = "lines",
            name = i[0]
        )
    )

In [44]:
fig.update_layout(
    title = "Flight Ratio in Different Time Sessions per Airport",
    xaxis_title = "Session",
    yaxis_title = "Ratio",
    legend_title = "Airport Code",
)